# 一、选题说明
## 1.项目背景
自2024年以来，中国南方多地出现强对流天气，部分地区突发短时强降水、雷暴大风等。专家表示，今年的强对流天气呈现早发、多发、强发的特征，如图 1所示，3月31日凌晨，江西省南昌市出现强对流天气，已造成4人死亡，10余人受伤。这次事件又一次警醒人们要格外关注强对流天气的预报，其持续带来的强降水叠加可能引发各类次生灾害，造成的城市内涝、山洪滑坡泥石流等地质灾害威胁到了人民的生命财产安全。
    
![](https://ai-studio-static-online.cdn.bcebos.com/28366b90fe9c43a789aaf32738edeca025fc102b41ad4c4aa1e22c12d9ab0756)

强对流天气是指发生突然、强度剧烈的对流性灾害天气过程，常伴有雷电、短时强降水、雷暴大风、冰雹、龙卷风等。虽然强对流天气空间尺度不大、生命周期短暂，却具有很强的破坏力，想要对其进行预报就需要抓住对流初生（Convective Initiation, CI）这一天气现象，对流初生的出现大概率意味着在未来30分钟至2小时内会有强对流天气的发生，如果能够准确观测出CI的发生时间、发生地点就能大大提高对流预报的精确度。因此，本项目旨在利用飞桨PaddlePaddle开发一个基于国产静止卫星数据的对流预报系统，以帮助减少强对流天气带来的灾难性后果。

## 2.为什么要做对流预报系统

近年来，我国发射的最新一代风云四号静止气象卫星可提供高时空分辨率成像资料、闪电监测资料、大气三维温湿度探测资料和空间天气监测资料，这些将在天气监测与预报、气象防灾减灾、应对气候变化、生态环境监测、航空和空间天气等领域得到广泛应用。早期的FY-4A作为风云四号静止气象卫星的首星发布过对流初生产品，为对流天气的监测提供了强有力的数据支撑。

但在今年1月23日，相关部门发布公告，风云四号A星已进入超寿命运行阶段。3月4日，实施A星从105°E漂移至86.5°E并暂停业务服务。

![](https://ai-studio-static-online.cdn.bcebos.com/1969c5ccec05442a93a480631c04b62a3567234114fd42ebab5b8b6fc64ee2eb)


与此同时，风云四号B星开始启用。风云四号B星在继承A星技术状态的基础上，进一步提升了大气变化识别精度、空间分辨率等有效载荷的观测性能，新增快速成像仪载荷可大幅提升了中小尺度灾害性天气预警预报能力。但在发布业务化产品时，我们发现不再提供对流初生产品，这可能是由于传统的阈值法虚报率较高。因此，在遥感大数据以及人工智能技术快速发展的背景下，急需开发新的对流初生监测系统。

我们拟通过使用历年的FY-4A的成像仪全圆盘4KML1数据，对应的FY-4ACI实时产品（全圆盘/标称）及其他辅助数据(卫星降水产品GPM-IMERG（Global Precipitation Measurement IMERG）来制作CI数据集，用于训练CI检测模型。通过分析FY-4A与FY-4B两颗卫星观测数据的差异性，将训练出的深度学习模型能够迁移到FY-4B上使用。用户使用时只需下载FY-4B的数据，放入模型中即可得到预测结果，不仅可以方便更多人的使用，还可以帮助我们更好地理解强对流天气的动态变化，从而采取相应的防范措施和优化规划，减少损失和保障社会经济发展。



## 3.需求分析

对流天气（如雷暴、龙卷风、冰雹等）对人们的日常生活和安全有重大影响。用户可从风云遥感数据服务网上获取数据，通过用户界面输入数据即可获得相应的CI标签，对于农业、渔业、航空、交通、科研等行业，准确的天气预报可以帮助做出更好的决策。深度学习技术的应用能够提供更为精准和可靠的数据支持，帮助这些行业优化决策和降低风险。

目前，有的机构单位还需要依赖CI的预测结果来进行生产和工作，现在却没有及时有效的途径准确获取到信息，我们提出了基于深度学习的CI预报模型，相比于传统的预报模式，我们用深度学习的方式，通过FY4A或FY4B提供的一级数据,就可以直接获得CI的检测结果，更加的高效安全和准确。

# 二、技术方案

![](https://ai-studio-static-online.cdn.bcebos.com/04f5a88bf2954a85962fb8878d6624ed712eeb433cf04cf4a9164fa20461ba7f)


## 工程结构

```
home
├── dataset
│   │   ├── 4a     # FY4A成像仪全圆盘4KML1数据
│   │   ├── 4anc    # FY4A对流初生实时产品(全圆盘/标称)
│   │   ├── 4b     # FY4B成像仪全圆盘4KML1数据
│   │   ├── cdata   # 预处理后数据
│   │   ├── clabel  # 预处理后标签
│   │   ├── GPM    # 卫星降水产品GPM-IMERG（Global Precipitation Measurement IMERG）
│   │   ├── hdf_out # FY4A 读取HDF文件写入npy文件存放文件夹
│   │   ├── nc_out  # FY4A 读取NC文件写入png文件存放文件夹
│   │   ├── ci_gpm  # CI标签结合GPM降水数据修正后存放文件夹
│   │   ├── test   # 测试集
│   │   ├── model  # 模型参数
│   │   ├── logs   # 日志
├── work
│   ├── config.ini          # 配置文件
│   ├── Data_processing_1.py   # 预处理代码1，2，3
│   ├── Data_processing_2.py
│   ├── Data_processing_3.py
│   ├── test.py            # 测试代码
│   ├── train.py           # 训练代码
│   ├── web.py            # 部署代码
```

配置文件

config.ini
```
[load]
area_extent = 70, 145, 10, 60.1 # 最小经度、最小纬度、最大经度、最大纬度
res = 0.05 # 分辨率

[dir]
hdf = /home/aistudio/dataset/4a
nc = /home/aistudio/dataset/4anc
hdf_out = /home/aistudio/dataset/hdf_out
nc_out = /home/aistudio/dataset/nc_out
gpm = /home/aistudio/dataset/GPM
ci_gpm = /home/aistudio/dataset/ci_gpm
data_dir = /home/aistudio/dataset/cdata
label_dir = /home/aistudio/dataset/clabel
model = /home/aistudio/dataset/model
logs = /home/aistudio/dataset/logs
test_dir = /home/aistudio/dataset/test
```

## 1.数据集收集与制作：

由于文件大小限制， 在aistudio上只上传一个时间节点的数据作为例子。

从风云遥感数据网下载2022-2023年的FY-4A的成像仪全圆盘4KML1数据，以及对应的FY-4ACI实时产品(全圆盘/标称)。对于成像仪全圆盘4KML1数据，取7-14通道数据，进行矫正裁剪，并归一化后作为模型训练的特征数据。对于标签数据集，由于传统的根据大数据统计设定多通道阈值范围，或者主观提取特征物理量进行对流判别的方法存在一定的局限性，所以FY-4ACI实时产品的Convective_Initiation子产品或存在一定的虚报，由于强对流天气常常伴有短时强降水，可以利用对应的卫星降水产品GPM-IMERG（Global Precipitation Measurement IMERG）进行比对，剔除虚报点。最终形成对流初生数据集。
在读取红外通道亮温数据时保留后8个通道，因为在查阅相关文献时发现CI的发生与后八个通道关系较大。最后切成128x128大小时剔除了标签为正较少的数据。

![](https://ai-studio-static-online.cdn.bcebos.com/74ba022422c34da0a9f34669a62c75cd43bce89444894fa78e67be70c8aa2982)

涉及代码为Data_processing_1.py, Data_processing_2.py, Data_processing_3.py.

## 2.模型的搭建、训练

![](https://ai-studio-static-online.cdn.bcebos.com/6e02771c509f4794ae5a6d7f9c5c46d6959aa169d5f740ed814cea866286bacd)

本项目的CI预报模型主要是采用分割网络通过学习多通道输入之间的非线性关系，来实现像素级的CI识别，整个CI预报网络采用的基本框架是U-HRNet，它的主要结构分为五个阶段，如图 4所示。主要包含以下两个功能：（1）从高分辨率表征中分离产生低分辨率分支的功能；（2）不同分辨率特征之间重复进行多尺度融合的功能。其中生成低分辨率分支的功能在阶段2-4的开头，重复的多尺度融合功能位于阶段2-4的末尾。

阶段1：网络从阶段1开始，输入的是FY-4A的8个红外通道数据。大小为256×256×8。首先经过两个3×3卷积层将特征维数从8扩展到32，同时增加了局部感受野。接着通过使用一个步长为2的3×3卷积来进行下采样，将特征图分辨率降低为128×128，从而达到减少计算量，提高检测效率的目的。然后通过一个瓶颈模块（Bottleneck block）来提高特征的多样性。Bottleneck block引入了非线性变换，进一步增加了网络的深度。最后，在阶段1中特征映射的输出大小为128×128×256。

阶段2：首先利用步长为1的3×3卷积对阶段1中提取的大小为128×128×256的特征图进行降维操作，得到一个的高分辨分支。同时，对128×128×256的特征图采用步长为2的3×3卷积进行下采样，生成一个低分辨率的分支。两个分支的特征图大小分别为128×128×32和64×64×64，从而实现不同分辨率特征的并行。然后，对每个分支中使用两个残差模块（Resblock）进行特征提取，增加了网络的深度。最后，将两个不同分辨率的分支进行多尺度融合，得到大小分别为128×128×32和64×64×64的特征图作为阶段2的输出。

阶段3和阶段4：与阶段2的结构类似. 

阶段5：首先，对阶段4输出的四个不同尺度的特征图进行融合，融合后的特征图大小为128×128×64。然后，对融合后的特征进行上采样：采用步长为2的反卷积，得到256×256×32的特征图，并使用U-Net网络的SC结构将此处的特征图与阶段1中提取的特征图进行特征映射。在最后一层，使用1×1卷积和sigmoid激活函数来映射所需类别，分为云和背景两类。

训练时损失函数选择Binary Cross Entropy Loss，优化器选择Adam，评价指标选择Intersection over Union。此外还使用了EarlyStopping，ReduceLROnPlateau，ModelCheckpoint等回调函数。

涉及代码为train.py, test.py.

## 3.web部署

编写好服务代码，将模型部署到基于gradio框架的web应用，用户可选择所提供的例子快速体验，在界面中还提供了一个风云卫星遥感数据服务网的账号，可点击相关按钮前往风云卫星遥感数据服务网获取需要的数据。在配置信息中还可以对经纬度范围，分辨率等参数进行修改。

![](https://ai-studio-static-online.cdn.bcebos.com/bd99154f659145f18b8c16b4c9cf6df70e0cb9b9a38c4791acafb49fc509dc60)

涉及代码为web.py.